In [ ]:
import pandas as pd
import numpy as np

# 1. Read data and weights
updated_df = pd.read_csv('nba_players_data_cleaned.csv')
weights_df_ahp = pd.read_csv('output_ranking_AHP.csv') 
weights_df_entropy = pd.read_csv('output_ranking_ENTROPY.csv') 

# 2. Merge AHP and entropy weights (you can adjust the merge method if necessary)
weights_df = pd.merge(weights_df_ahp, weights_df_entropy, on='Criteria')
weights_df['Final Weight'] = (weights_df['AHP Weight'] + weights_df['Entropy Weight']) / 2

# 3. Data normalisation
normalized_cols = ['pts', 'reb', 'ast', 'usg_pct'] 
for col in normalized_cols:
    updated_df[col] = updated_df[col] / updated_df[col].max()

# 4. Calculation of weighted values
for col in normalized_cols:
    updated_df[f'{col}_weighted'] = updated_df[col] * weights_df[weights_df['Criteria'] == col]['Final Weight'].values[0]

# 5. Calculate of ideal best and worst solutions
ideal_best = updated_df[[f'{col}_weighted' for col in normalized_cols]].max()
ideal_worst = updated_df[[f'{col}_weighted' for col in normalized_cols]].min()

# 6. Calculate Euclidean distances
updated_df['S_i+'] = np.sqrt(((updated_df[[f'{col}_weighted' for col in normalized_cols]] - ideal_best) ** 2).sum(axis=1))
updated_df['S_i-'] = np.sqrt(((updated_df[[f'{col}_weighted' for col in normalized_cols]] - ideal_worst) ** 2).sum(axis=1))

# 7. Calculate TOPSIS scores
updated_df['TOPSIS_score'] = updated_df['S_i-'] / (updated_df['S_i+'] + updated_df['S_i-'])

# 8. Sort players by TOPSIS score
updated_df.sort_values(by='TOPSIS_score', ascending=False, inplace=True)

# 9. Save TOPSIS rankings to a CSV file
updated_df[['player_name', 'TOPSIS_score']].to_csv('output_ranking_TOPSIS.csv', index=False)

print("Classement TOPSIS sauvegardé dans 'output_ranking_TOPSIS.csv'")
